In [1]:
import pandas as pd
import numpy as np
from utils import get_species, get_labels, get_labels_all

In [57]:
X, y, y_all = get_species(), 1.0 * get_labels(), get_labels_all()

In [58]:
cols = ((X > 0.00001).mean(axis=0) > 0.01).values
cols

array([ True,  True,  True, ..., False, False,  True])

In [59]:
X = X.iloc[:, cols]
X.shape

(12532, 542)

In [60]:
X = X[list(filter(lambda x : "virus" not in x and "Virus" not in x and "unclassified" not in x, X.columns))]
X.shape

(12532, 413)

In [61]:
remove = ((y_all == "Underweight") | (y_all == "Overweight") | (y_all == "Obesity")).values

In [62]:
X, y, y_all = X.iloc[~remove, :], y.iloc[~remove, :], y_all.iloc[~remove, :]

In [63]:
y_all

PHENOTYPE_Disease
Study_ID Sample Accession                    
GMHI-23  SAMEA3879547                 Healthy
         SAMEA3879551                 Healthy
         SAMEA3879543                 Healthy
         SAMEA3879565                 Healthy
         SAMEA3879546                 Healthy
...                                       ...
P140     SAMN07509557      Ulcerative colitis
         SAMN07509558      Ulcerative colitis
         SAMN07509546      Ulcerative colitis
         SAMN07509552      Ulcerative colitis
         SAMN07509921      Ulcerative colitis

[10039 rows x 1 columns]

In [64]:
studies = np.unique(X.index.get_level_values(0))
studies

array(['GMHI-10', 'GMHI-11', 'GMHI-13', 'GMHI-14', 'GMHI-15', 'GMHI-17',
       'GMHI-19', 'GMHI-2', 'GMHI-21', 'GMHI-23', 'GMHI-24', 'GMHI-26',
       'GMHI-27', 'GMHI-28', 'GMHI-3', 'GMHI-31', 'GMHI-32', 'GMHI-33',
       'GMHI-4', 'GMHI-5', 'GMHI-9', 'GMHI-V-35', 'GMHI-V-36',
       'GMHI-V-38', 'GMHI-V-40', 'GMHI-V-41', 'P103', 'P11', 'P110',
       'P113', 'P121', 'P13', 'P132', 'P135', 'P136', 'P140', 'P15',
       'P17', 'P2', 'P21', 'P24', 'P32', 'P34', 'P39', 'P4', 'P47', 'P48',
       'P53', 'P56', 'P57', 'P59', 'P63', 'P69', 'P74', 'P76', 'P77',
       'P8', 'P80', 'P81', 'P86', 'P87', 'P88', 'P89', 'P9', 'P94', 'P95',
       'P96', 'P98'], dtype=object)

In [65]:
np.random.seed(42)
perm = np.random.permutation(len(studies))
prop = 0.9
train_idx, test_idx = perm[:int(len(studies) * prop)], perm[int(len(studies) * prop):]
train_studies = studies[train_idx]
test_studies = studies[test_idx]

In [66]:
c = 0.00001
X_train, y_train = (X.loc[train_studies].values > c) * 1.0, y.loc[train_studies].values * 1.0
X_test, y_test = (X.loc[test_studies].values > c) * 1.0, y.loc[test_studies].values * 1.0

In [67]:
X_train.shape, X_test.shape, y_train.shape

((9011, 413), (1028, 413), (9011, 1))

In [68]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score

clf = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", C=1, class_weight="balanced")
clf.fit(X_train, y_train[:, 0])
y_hat = clf.predict(X_test)
balanced_accuracy_score(y_test[:, 0], y_hat)

0.6509337732247948

In [69]:
import torch
import math

ModuleNotFoundError: No module named 'torch'

In [70]:
dtype = torch.float
device = torch.device("cpu")

NameError: name 'torch' is not defined

In [372]:
X_train_t = torch.from_numpy(X_train).type(torch.FloatTensor)
X_test_t = torch.from_numpy(X_test).type(torch.FloatTensor)
y_train_t = torch.from_numpy(y_train).type(torch.FloatTensor)
y_test_t = torch.from_numpy(y_test).type(torch.FloatTensor)
y_train_t

tensor([[1.],
        [1.],
        [1.],
        ...,
        [0.],
        [0.],
        [0.]])

In [376]:
model = torch.nn.Sequential(
    torch.nn.Linear(413, 30),
    torch.nn.ReLU(),
    torch.nn.Linear(30, 1),
    torch.nn.Sigmoid(),
)

In [377]:
# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum')

In [378]:
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
for t in range(2000):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(X_train_t)

    # Compute and print loss.
    loss = loss_fn(y_pred, y_train_t)
    if t % 100 == 0:
        print(t, loss.item())
        pred = (model(X_train_t))[:, 0] > 0.5
        print(balanced_accuracy_score(pred, y_train_t[:, 0]))
        pred = (model(X_test_t))[:, 0] > 0.5
        print(balanced_accuracy_score(pred, y_test_t[:, 0]))

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

0 2290.1171875
0.4704166622214331
0.5107226520207091
100 1246.9920654296875
0.7840559323934309
0.6483843401408418
200 1138.917236328125
0.8093310101144151
0.6549075391180654
300 1023.1218872070312
0.8369544948222478
0.6593700944901748
400 859.9425048828125
0.8785113084218994
0.6571562308098615
500 682.5897216796875
0.9179660222646457
0.6473539290402878
600 525.6119995117188
0.9460472818757484
0.6352580602474954
700 399.13861083984375
0.9660768301761189
0.6183035054922332
800 301.90313720703125
0.9790474616760219
0.6183035054922332


KeyboardInterrupt: 